In [1]:
import pandas as pd

# Фракционный счёт по данным WoS

В данном документе проиллюстрированы механики, которые позволяют посчитать один из показателей программы «Приоритет
2030», а именно:

> Количество публикаций в научных изданиях I и II квартилей, а также научных изданиях, включенных в индексы Arts and
> Humanities Citation Index (A&HCI) и Book Citation Index - Social Sciences & Humanities (BKCI-SSH), индексируемых в
> базе данных Web of Science Core Collection, в расчете на одного научно-педагогического работника (далее — НПР)

Исходный приказ *(актуален на момент создание данного документа 11.07.2021)*, в котором фигурирует показатель можно
найти по [ссылке](http://publication.pravo.gov.ru/Document/View/0001202106230024), в т.ч. в документе описана методика
расчёта показателя.

## Уточнение задачи

1. Для примера возьмём один из вузов — [Университет ИТМО](https://itmo.ru/) (почему именно этот университет я упомяну ниже в блоке про IT конференции)
1. 3 года для анализа — 2020, 2019, 2018
1. Используем относительно легко доступные данные — интерфейсы и выгрузки WoS доступные университетам по нац. подписке

## Подготовка данных для анализа

### Аккаунт университета в WoS

**Обратите внимание!** На момент написания WoS обновляло свои интерфейсы и последующие инструкции могут не сработать в
полном объёме через какое-то время.

1. Переходим к [интерфейсу поиска](https://apps.webofknowledge.com/WOS_GeneralSearch_input.do?product=WOS&search_mode=GeneralSearch&SID=E2IlyQDP2bZHZP9dgMJ&preferencesSaved=)
1. Далее переключаемся на [«Расширенный поиск»](https://apps.webofknowledge.com/WOS_AdvancedSearch_input.do?SID=E2IlyQDP2bZHZP9dgMJ&product=WOS&search_mode=AdvancedSearch)
1. В блоке обозначение полей ищем «OG= Профили организаций» и переходим по ссылке «[Указатель]».
1. В блоке поиска вбиваем какую-то часть официального названия университета на английском языке и наживаем на «Find» (в
моём случае хватило «ITMO»).
1. В списке найденного находим нашу организацию и нажав на «D» в гругляшке (старый UI WoS конечно «шикарен» 🤦‍),
переходим к расширенному списку названий организации.
1. Нажимаем на «Add» рядом с «Organization Name», что добавит основное название организации UI элемент, где написано
«Transfer your selected organization(s) below to the Organizations - Enhanced field on the search page.» и больше здесь,
пока, **ничего не нажимайте**.
1. Далее формируем исчерпывающий список названий университета для фракционного счёта, чтобы не возвращаться больше на
данную страницу.

In [2]:
names = [
    'ITMO UNIVERSITY',
    'SPBNRU ITMO',
    'SAINT PETERSBURG STATE UNIVERSITY OF INFORMATION TECHNOLOGIES, MECHANICS',
    'ITMO UNIV',
    'ST PETERSBURG ST UNIV IT MECH OPT',
    'DF USTINOV ST PETERSBURG STATE TECH UNIV VOENMEH',
    'FED ST PETERSBURG UNIV INFORMAT TECHNOL',
    'FED UNIV INFORMAT TECHNOL MECH OPT',
    'FINE MECH OPT INST',
    'FINE MECH OPT INST ST PETERSBURG',
    'FINE MECH OPTICS INST',
    'INFORMAT TECHNOL MECH OPT ITMO UNIV',
    'INST FINE MECH OPT',
    'INST FINE MECH OPT LASER CTR',
    'INST FINE MECH OPT ST PETERSBURG',
    'INST FINE MECH OPTICS',
    'INST PRECIS MECH OPT',
    'INST PRECIS MECH OPT TECH UNIV',
    'INST PRECISION MECH OPTICS',
    'INST PRECISION MECHAN OPT',
    'ITMO',
    'ITMO NATL RES UNIV',
    'ITMO ST PETERSBURG',
    'ITMO UNIV',
    'ITMO UNIV 197101',
    'ITMO UNIV ST PETERSBURG',
    'LENINGRAD FINE MECH OPT INST',
    'LENINGRAD FINE MECH OPT IST',
    'LENINGRAD FINE MECH OPT RES INST',
    'LENINGRAD FINE MECH OPTIC INST',
    'LENINGRAD FINE MECH OPTICS INST',
    'LENINGRAD FINE TECH OPTICS INST',
    'LENINGRAD HIGH PRECIS MECH OPT INST',
    'LENINGRAD INST FINE MECH OPT',
    'LENINGRAD INST PREC MECH OPT',
    'LENINGRAD INST PRECIS MECH ENGN OPT',
    'LENINGRAD INST PRECIS MECH OPT',
    'LENINGRAD INST PRECISE MECH OPT',
    'LENINGRAD INST PRECISION MECH OPTICS',
    'LENINGRAD OPT INST',
    'LENINGRAD PREC MECH OPT INST',
    'LENINGRAD PRECIS ENGN OPT INST',
    'LENINGRAD PRECIS MECH OPT',
    'LENINGRAD PRECIS MECH OPT INST',
    'LENINGRAD PRECIS MECH OPT RES INST',
    'LENINGRAD PRECIS MECH OPTICS INST',
    'LENINGRAD PRECISE MECH OPT',
    'LENINGRAD PRECISE MECH OPT INST',
    'LENINGRAD PRECISE MECH OPTICS INST',
    'LENINGRAD PRECISION MECH OPTICS INST',
    'LENINGRAD PRESIC OPTICS MECH INST',
    'NAT RES UNIV INFORMAT TECHNOL MECH OPT ITMO',
    'NATL RES INST ITMO',
    'NATL RES UNIV INFORMAT TECHNOL',
    'NATL RES UNIV INFORMAT TECHNOL MECH OPT',
    'NATL RES UNIV INFORMAT TECHNOL MECH OPT ITMO',
    'NATL RES UNIV INFORMAT TECHNOL MECH OPT ITMO UN',
    'NATL RES UNIV INFORMAT TECHNOL MECH OPT NRU ITM',
    'NATL RES UNIV INFORMAT TECHNOL MECH OPT ST PETE',
    'NATL RES UNIV INFORMAT TECHNOL MECH OPT UNIV IT',
    'NATL RES UNIV ITMO',
    'NATL STATE UNIV INFORMAT TECHNOL MECH OPT',
    'NATL UNIV INFORMAT TECHNOL MECH OPT ITMO',
    'NRU ITMO',
    'NRU ITMO NATL RES UNIV INFORMAT TECHNOL MECH OPT',
    'PETERSBURG STATE INST FINE MECH OPT',
    'PREC MECH OPT INST',
    'PRECIS MECH OPT INST',
    'PRECISE MECH OPTICS INST',
    'PRECISION MECH OPTICS INST',
    'RES CTR INFORMAT OPT TECHNOL ITMO UNIV',
    'RES INST ITMO',
    'RES UNIV INFORMAT TECHNOL MECH OPT',
    'RES UNIV ITMO',
    'RUSSIAN PRECIS MECH OPT STATE INST',
    'SBBSU ITMO',
    'SPB NATL RES UNIV INFORMAT TECHNOL MECH OPT',
    'ST PB UNIV IT MECH OPT',
    'ST PETERBURG NATL RES UNIV INFORMAT TECHNOL MECH',
    'ST PETERBURG OPT INST',
    'ST PETERBURG OPT MAT SCI RES TECHNOL INST',
    'ST PETERBURG PRECIS MECH OPT INST',
    'ST PETERBURG PRECISE MECH OPT INST',
    'ST PETERBURG STATE UNIV INFORMAT TECHNOL MECH O',
    'ST PETERSBERG PRECIS MECH OPT INST',
    'ST PETERSBRUG PRECIS MECH OPT INST',
    'ST PETERSBUG INST FINE MECH OPT',
    'ST PETERSBURG FED INST FINE MECH OPT',
    'ST PETERSBURG FINE MECH OPT INST',
    'ST PETERSBURG GOVT INST FINE MECH OPT',
    'ST PETERSBURG INST FINE MECH OPT',
    'ST PETERSBURG INST FINE MECH OPT TECH UNIV',
    'ST PETERSBURG INST FIRE MECH OPT',
    'ST PETERSBURG INST PRECIS MECH OPT',
    'ST PETERSBURG INST PRECISE MECH OPT',
    'ST PETERSBURG INST SHARP MECH OPT',
    'ST PETERSBURG ITMO UNIV',
    'ST PETERSBURG NATL RES UNIV INFORMAT TECH MECH',
    'ST PETERSBURG NATL RES UNIV INFORMAT TECHNOL',
    'ST PETERSBURG NATL RES UNIV INFORMAT TECHNOL MECH',
    'ST PETERSBURG NATL RES UNIV ITMO',
    'ST PETERSBURG NATL UNIV INFORMAT TECHNOL MECH O',
    'ST PETERSBURG PRECIS MECH OPT INST',
    'ST PETERSBURG PRECIS MECH OPT STATE INST',
    'ST PETERSBURG PRECISE MECH OPT INST',
    'ST PETERSBURG RES UNIV INFORMAT TECHNOL MECH OP',
    'ST PETERSBURG SCI RES UNIV INFORMAT TECHNOL MECH',
    'ST PETERSBURG STATE FINE MECH OPT INST',
    'ST PETERSBURG STATE INST FINE MECH OPT',
    'ST PETERSBURG STATE INST FINE MECH OPT TECH UNI',
    'ST PETERSBURG STATE INST INFORMAT TECHNOL MECH',
    'ST PETERSBURG STATE INST PRECIS MECH OPT',
    'ST PETERSBURG STATE INST PRECIS MECH OPT ENGN U',
    'ST PETERSBURG STATE INST PRECIS MECH OPT TECH U',
    'ST PETERSBURG STATE INST PRECISE MECH OPT',
    'ST PETERSBURG STATE PRECIS MECH OPT INST',
    'ST PETERSBURG STATE TECH UNIV INFORMAT TECHNOL ME',
    'ST PETERSBURG STATE UNIV FINE MECH OPT',
    'ST PETERSBURG STATE UNIV INFOMAT TECHNOL MECH O',
    'ST PETERSBURG STATE UNIV INFORM TECHNOL MECH OP',
    'ST PETERSBURG STATE UNIV INFORM TECHNOL MECH OPT',
    'ST PETERSBURG STATE UNIV INFORMAT MECH OPT',
    'ST PETERSBURG STATE UNIV INFORMAT MECH OPT TECH',
    'ST PETERSBURG STATE UNIV INFORMAT TECHNOL',
    'ST PETERSBURG STATE UNIV INFORMAT TECHNOL FINE ME',
    'ST PETERSBURG STATE UNIV INFORMAT TECHNOL MECH',
    'ST PETERSBURG STATE UNIV INFORMAT TECHNOL MECH O',
    'ST PETERSBURG STATE UNIV INFORMAT TECHNOL MECH OP',
    'ST PETERSBURG STATE UNIV INFORMAT TECHNOL MECHAN',
    'ST PETERSBURG STATE UNIV INFORMAT TECHNOL MED O',
    'ST PETERSBURG STATE UNIV INFORMAT TECHNOLOGIES',
    'ST PETERSBURG STATE UNIV INFORMAT TECHNOLOGIES M',
    'ST PETERSBURG STATE UNIV INFORMAT TECHOL MECH O',
    'ST PETERSBURG STATE UNIV INFORMATON TECHNOL MECH',
    'ST PETERSBURG STATE UNIV INNOVAT TECHNOL MECH O',
    'ST PETERSBURG STATE UNIV ITMO',
    'ST PETERSBURG STATE UNIV LOW TEM FOOD TECHNOL',
    'ST PETERSBURG STATE UNIV LOW TEMP',
    'ST PETERSBURG STATE UNIV LOW TEMP FOOD TECHNOL',
    'ST PETERSBURG STATE UNIV LOW TEMP FOODSTUFF TEC',
    'ST PETERSBURG STATE UNIV LOW TEMP TECHNOL',
    'ST PETERSBURG STATE UNIV REFRIGERAT',
    'ST PETERSBURG STATE UNIV REFRIGERAT FOOD ENG',
    'ST PETERSBURG STATE UNIV REFRIGERAT FOOD ENGN',
    'ST PETERSBURG STATE UNIV REFRIGERAT FOOD TECHNO',
    'ST PETERSBURG TECH UNIV ITMO',
    'ST PETERSBURG UNIV FINE MECH OPT',
    'ST PETERSBURG UNIV INFORMAT SCI MECH OPT',
    'ST PETERSBURG UNIV INFORMAT TECHNOL',
    'ST PETERSBURG UNIV INFORMAT TECHNOL MECH OPT',
    'ST PETERSBURG UNIV INFORMAT TECHNOL MECH OPT IT',
    'ST PETERSBURG UNIV INFORMAT TECHNOL OPT',
    'ST PETERSBURG UNIV ITMO',
    'ST PETERSBURG UNIV REFRIGERAT FOOD ENGN',
    'ST PETERSPURG UNIV INFORMAT TECHNOL MECH OPT',
    'STATE INST FINE MECH OPT',
    'STATE INST INFORMAT TECHNOL MECH OPT',
    'STATE INST PRECIS MECH OPT',
    'STATE OPT INST',
    'STATE PETERSBURG STATE UNIV INFORMAT TECHNOL MECH',
    'STATE UNIV FINE MECH OPT',
    'STATE UNIV INFORMAT TECHNOL',
    'STATE UNIV INFORMAT TECHNOL MECH OPT',
    'STATE UNIV INFORMAT TECHNOL MECH OPT ST PETERSB',
    'STPETERSBURG STATE INST PRECIS MECH OPT',
    'STR PETERSBURG STATE UNIV INFORMAT TECHNOL MECH',
    'TECH UNIV INFORMAT TECHNOL MECH OPT',
    'UNIV INFORMAT TECHNOL FINE MECH OPT',
    'UNIV INFORMAT TECHNOL MECH OPT',
    'UNIV INFORMAT TECHNOL MECH OPT ITMO',
    'UNIV ITMO',
    'UNIV ITMO NATL RES UNIV',
    'UNIV LOW TEMPERATURE TECHNOL',
    'UNP LASER CTR ITMO'
]

8. В завершении нажимаем «Ok» внизу страницы и переходим к поиску публикаций по организации.

---

Тем, кто знаком с консолью DevTools браузера (например в Chrome открывается по `F12`) упростит формирование списка
названий из **пункта № 7**, если вы введёте и запустите ниже представленный код:

```js
$('#SAOuterTable > tbody > tr:nth-child(1) > td > table > tbody > tr > td > table:nth-child(2) > tbody > tr > td > table > tbody > tr:nth-child(2) > td.SAContent > table tr').each(function() {
    console.log($(this).find('td:last > span').html())
})
```

В консоль выведется список, который можно скопировать в любой текстовый редактор и довольно быстро почистить с помощью
функции «Найти и заменить». Останется только вручную взять варианты названия из «Other Names».

---

### Выгрузка публикаций

Если в предыдущем блоке +/- всё было сделано правильно Вас должно перебросить на страницу расширенного поиска, с
указанным профилем организации (в моём случае `OG=(ITMO University)`).

1. Нажимаем кнопку «Поиск».
1. Ниже на странице в «Истории поиска» появится подборка с каким-то количеством результатов (в моём случае было найдено
15 563 документа).
1. Далее **Министерство придумало для нас квест**, который вынуждает нас не просто выгрузить выдачу полученную в предыдущем
пункте, а формировать несколько выборок с помощью фильтров и выгружать их.

Прежде чем мы перейдём к описанию решения квеста, опишем общие правила выгрузок, которые мы будем воспроизводить каждый
раз после решения части загадок (формирования удовлетворяющих выборок):

1. Нажимаем кнопку «Экспорт»
1. В выпадающем меню выбираем «Другие форматы файлов»
1. В сплывающем окне вводим следующие данные:
    * Записи с *такой-то* по *такую-то* (максимально за раз можно выгрузить по 500 записей, возможно Вам придётся
    сделать это несколько раз, что бы сформировать весь объём Ваших публикаций)
    * «Содержимое записи» устанавливаем в «Полная запись»
    * «Формат файла» выбираем «С разделителями табуляции (Win, UTF-8)»
1. Нажимаем на кнопку «Экспорт» и сохраняем файл в папку `2021_07_07_wos_q1_q2_with_fractions/data` данного проекта
(чтобы было понятней, рекомендую сохранять файлы по следующему шаблону `[publication_type]-[year]-[start]-[end].csv`,
например для научных статей для 2020 года название файла может выглядеть так:
`articles-and-reviews-2020-1-500.csv`)

#### Статьи и обзоры за 3 года

Работаем с блоком «Уточнение результатов» в нашей исходной выдаче по организации:

1. Нажимаем «Просмотреть все параметры» внизу блока «Уточнение результатов».
1. В блоке «Web of Science Index» нажимаем «дополнительные параметры / значения...».
1. В открывшемся интерфейсе проставляем галочки рядом с:
    * Science Citation Index Expanded
    * Social Sciences Citation Index
    * Arts and Humanities Citation Index
1. Нажимаем кнопку «Уточнить». Вас перебросит обратно сузив результаты выдачи.
1. Ставим галочки напротив интересующих нас годов (2020, 2019, 2018) и нажимаем кнопку «Уточнить»
1. В блоке «Типы документов» ставим галочки напротив «ARTICLE» и «REVIEW», и наживаем на кнопку «Уточнить»

**Обратите внимание!** В процессе сужения результатов поиска по Вашему университету может отличаться
количество/вариативность сущностей с описанным выше. Это связано с тем, что система даёт работать, после каждого шага,
с сущностями, в которых есть статьи с аффилиацией Вашего университета.

**Ура!** Мы сформировали первую выборку для работы 😊

**Есть альтернативный вариант**, который мне более приятен, т.к. меньше переходов по интерфейсу и нажатий на кнопочки.
Но было важно показать все варианты использования интерфейса WoS.

1. Переходим на страницу [расширенного поиска](https://apps.webofknowledge.com/WOS_AdvancedSearch_input.do?SID=E2IlyQDP2bZHZP9dgMJ&product=WOS&search_mode=AdvancedSearch).
1. Вставляем в строку поиска следующий запрос:
```
OG=(ITMO University)
```
1. В блоке «Ограничить результаты по языкам и типам документов» с зажатым `Ctrl` отмечаем «Article» и «Review»,
дискриминацию по языкам не устраиваем.
1. В блоке «Период» выбираем «Настраиваемый диапазон лет» и с 2018 по 2020.
1. Выведем дополнительные настройки нажав на «ДУРУГИЕ ПАРАМЕТРЫ»
1. Web of Science Core Collection: указатели цитирования оставляем следующие галочки:
    * Science Citation Index Expanded (SCI-EXPANDED) --1975-по настоящее время
    * Social Sciences Citation Index (SSCI) --1975-по настоящее время
    * Arts & Humanities Citation Index (A&HCI) --1975-по настоящее время
1. Нажимаем на кнопку «Поиск» и переходим к результатам поиска.

У [выбранного мной университета](http://itmo.ru/) в конечном счёте осталось 3 207 публикаций удовлетворяющих
описанным выше ограничениям и фильтрам. Результаты экспортированы в файлы:

* `2021_07_07_wos_q1_q2_with_fractions/data/articles-and-reviews-2020-2018-1-500.csv`
* `2021_07_07_wos_q1_q2_with_fractions/data/articles-and-reviews-2020-2018-501-1000.csv`
* `2021_07_07_wos_q1_q2_with_fractions/data/articles-and-reviews-2020-2018-1001-1500.csv`
* `2021_07_07_wos_q1_q2_with_fractions/data/articles-and-reviews-2020-2018-1501-2000.csv`
* `2021_07_07_wos_q1_q2_with_fractions/data/articles-and-reviews-2020-2018-2001-2500.csv`
* `2021_07_07_wos_q1_q2_with_fractions/data/articles-and-reviews-2020-2018-2501-3000.csv`
* `2021_07_07_wos_q1_q2_with_fractions/data/articles-and-reviews-2020-2018-3001-3500.csv`

### Книжки за 3 года

1. Переходим на страницу [расширенного поиска](https://apps.webofknowledge.com/WOS_AdvancedSearch_input.do?SID=E2IlyQDP2bZHZP9dgMJ&product=WOS&search_mode=AdvancedSearch).
1. Вставляем в строку поиска следующий запрос:
```
OG=(ITMO University)
```
1. В блоке «Ограничить результаты по языкам и типам документов» с зажатым `Ctrl` отмечаем «Book»,
дискриминацию по языкам не устраиваем.
1. В блоке «Период» выбираем «Настраиваемый диапазон лет» и с 2018 по 2020.
1. Web of Science Core Collection: указатели цитирования оставляем следующие галочки:
    * Book Citation Index– Social Sciences & Humanities (BKCI-SSH) --2005-по настоящее время
1. Нажимаем на кнопку «Поиск» и переходим к результатам поиска.

К сожалению у [ИТМО](http://itmo.ru/) таких документов не оказалось, экспортировать нечего 🤷

### Бонусная IT-выборка

Данная выборка далась тяжело, возможно мне не повезло, но пришлось перебрать 7-10 университетов, прежде чем в выдаче
появились публикации, на которых можно было проиллюстрировать механику подсчёта показателя (даже вузы из
[5-100](https://www.5top100.ru/) не подходили).

1. Переходим на страницу [расширенного поиска](https://apps.webofknowledge.com/WOS_AdvancedSearch_input.do?SID=E2IlyQDP2bZHZP9dgMJ&product=WOS&search_mode=AdvancedSearch).
1. Вставляем в строку поиска следующий запрос:
```
OG=(ITMO University) AND
WC=(COMPUTER SCIENCE INFORMATION SYSTEMS OR COMPUTER SCIENCE ARTIFICIAL INTELLIGENCE OR COMPUTER SCIENCE THEORY METHODS OR COMPUTER SCIENCE INTERDISCIPLINARY APPLICATIONS OR COMPUTER SCIENCE SOFTWARE ENGINEERING OR COMPUTER SCIENCE CYBERNETICS OR COMPUTER SCIENCE HARDWARE ARCHITECTURE)
```
1. В блоке «Ограничить результаты по языкам и типам документов» с зажатым `Ctrl` отмечаем «Proceedings Paper»,
дискриминацию по языкам не устраиваем.
1. В блоке «Период» выбираем «Настраиваемый диапазон лет» и с 2018 по 2020.
1. Web of Science Core Collection: указатели цитирования оставляем следующие галочки:
    * Conference Proceedings Citation Index- Science (CPCI-S) --1990-по настоящее время
1. Нажимаем на кнопку «Поиск» и переходим к результатам поиска.

У меня найдено 533 документа, результаты экспортированы в файлы:

* `2021_07_07_wos_q1_q2_with_fractions/data/it-conf-2020-2018-1-500.csv`
* `2021_07_07_wos_q1_q2_with_fractions/data/it-conf-2020-2018-501-1000.csv`

**Отлично!** Сформирован комплект публикаций. Давайте посмотрим на пример данных (выведены первые 5 записей).

In [8]:
df = pd.read_csv('data/articles-and-reviews-2020-2018-1-500.csv', sep='\t').reset_index(drop=True)
df.head()

,PT,AU,BA,BE,GP,AF,BF,CA,TI,SO,...,PG,WC,SC,GA,UT,PM,OA,HC,HP,DA
0,"Sedov, DD; Kozin, VK; Iorsh, IV",NaN,NaN,NaN,"Sedov, D. D.; Kozin, V. K.; Iorsh, I., V",NaN,NaN,Chiral Waveguide Optomechanics: First Order Qu...,PHYSICAL REVIEW LETTERS,NaN,...,"Physics, Multidisciplinary",Physics,PN1MF,WOS:000604249900009,33449725.0,NaN,NaN,NaN,2021-07-11,NaN
1,"Bobylev, DA; Smirnova, DA; Gorlach, MA",NaN,NaN,NaN,"Bobylev, Daniel A.; Smirnova, Daria A.; Gorlac...",NaN,NaN,Photonic Topological States Mediated by Stagge...,LASER & PHOTONICS REVIEWS,NaN,...,"Optics; Physics, Applied; Physics, Condensed M...",Optics; Physics,QK1XH,WOS:000603121800001,NaN,NaN,NaN,NaN,2021-07-11,NaN
2,"Lorenz-Meyer, N; Bobtsov, A; Ortega, R; Nikola...",NaN,NaN,NaN,"Lorenz-Meyer, Nicolai; Bobtsov, Alexey; Ortega...",NaN,NaN,PMU-based decentralised mixed algebraic and dy...,IET GENERATION TRANSMISSION & DISTRIBUTION,NaN,...,"Engineering, Electrical & Electronic",Engineering,QJ4AL,WOS:000619633900023,NaN,NaN,NaN,NaN,2021-07-11,NaN
3,"Finkelstein, M; Cha, JH; Ghosh, S",NaN,NaN,NaN,"Finkelstein, Maxim; Cha, Ji Hwan; Ghosh, Shyamal",NaN,NaN,On dynamic information-based life extension,PROCEEDINGS OF THE INSTITUTION OF MECHANICAL E...,NaN,...,"Engineering, Multidisciplinary; Engineering, I...",Engineering; Operations Research & Management ...,TB2IJ,WOS:000637903600001,NaN,NaN,NaN,NaN,2021-07-11,NaN
4,"Barnett, R; Westbury, MV; Sandoval-Velasco, M;...",NaN,NaN,NaN,"Barnett, Ross; Westbury, Michael V.; Sandoval-...",NaN,NaN,Genomic Adaptations and Evolutionary History o...,CURRENT BIOLOGY,NaN,...,Biochemistry & Molecular Biology; Biology; Cel...,Biochemistry & Molecular Biology; Life Science...,PN6ZE,WOS:000604624500007,33065008.0,"Other Gold, Green Published",NaN,NaN,2021-07-11,NaN


Название колонок сходу ничего не говорит, путём визуального анализа выяснилось следующее:

* **AU** — авторы
* **C1** — список аффилиаций с привязанными к ним авторами
* **SN** — ISSN
* **EI** — eISSN
* **PY** — год публикации
* **DI** — DOI
* **CT** - название конференции

На всё остальное в данной аналитике можно забить, хватит вышеупомянутых столбцов.

## Квартильность публикаций

В исходном квесте от министерства есть ограничения, что статьи должны быть I и II квартиля. Фильтр по квартилю к книгам
и публикациям в конференциях я не применял, т.к. количество этих объектив у университетов и так катастрофически мало.

Нам потребуется таблица, которая будет содержать ISSN и eISSN журналов, а также их квартиль. Данные можно взять с
[данной страницы](https://jcr.clarivate.com/jcr/browse-journals), предварительно применив следующие фильтры:

* В «Citation Indexes» оставляем только интересующие нас индексы:
    * Science Citation Index Expanded (SCIE)
    * Social Science Citation Index (SSCI)
    * Arts & Humanities Citation Index (AHCI)
* В «JIF Quartile» ставим галочки напротив «Quartile 1» и «Quartile 2»
* «JCR Year» последовательно выставляем по годам: 2020, 2019, 2018

У меня получилось 6 678, 6 372, 6 169 журналов за 2020, 2019 и 2018 года соответственно. Если вы не знакомы с DevTools
браузера придётся копировать данные в ручную, но если таковые знания есть обратите внимание на вкладку «Network» и тип
прилетающих данных со страницы «XHR».  Это существенно ускорит процесс формирования необходимых данных для аналитики 😏

У меня получилось что-то вроде такого (выведены первые 5 записей):

In [13]:
df = pd.read_csv('data/wos-journals-q1-q2.csv', sep=',')
df.head()

,jcrYear,issn,eissn,quartile
0,2018,0007-9235,1542-4863,Q1
1,2018,2058-8437,2058-8437,Q1
2,2018,0028-4793,1533-4406,Q1
3,2018,0140-6736,1474-547X,Q1
4,2018,1474-1776,1474-1784,Q1


## Фракционный счёт

Будем считать методом фракционного счёта полученные данные в следующем порядке:

1. «Articles» and «Reviews» из индексов SCI-EXPANDED, SSCI и A&HCI
2. «Books» считать по [ИТМО](http://itmo.ru/) нечего 🤷‍
3. IT-конференции

Одновременно будем проставлять признак `is_good` когда объект полностью удовлетворяет условиям показателя программы.